<a href="https://colab.research.google.com/github/sseun43/Langchain_RAG_Demo/blob/main/Using_Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain langchain_community langchain_chroma

In [ ]:
pip install -qU langchain-openai

In [3]:
import getpass
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
%pip install --upgrade --quiet pypdf

In [ ]:
pip install langchainhub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "/content/drive/MyDrive/Speech_processing.pdf" # A pdf file about sounds and speeches
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split(text_splitter)


vectorstore = Chroma.from_documents(documents=pages, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What kind of classification is south african zulu language?")

'The South African Zulu language is classified as a click language, which features non-pulmonic sounds like clicks. This distinguishes it from many other languages that primarily use pulmonic sounds. Additionally, Zulu is part of the Nguni group of Bantu languages.'

In [9]:
# cleanup
vectorstore.delete_collection()